# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import mpl_toolkits.mplot3d

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
#import inauguralproject
from HouseholdSpecializationModel import  HouseholdSpecializationModelClass



In [ ]:
model = HouseholdSpecializationModelClass()
model.estimate(do_print=True)


# Question 2

plotting $Wf/Wm$ ratio

# Question 3

Plot the $Wf / Wm$ ratio for the continuous values 

# Question 4

Choose $\alpha$ and $\sigma$ such that it minimizes the expression. Illustrate how the model fit the data and discuss the economic interpretation of the results.

# Question 5

ADD CONCISE CONLUSION.